# Getting New Data

In [ ]:
import numpy as np
import pandas as pd
import requests
import prepare
from time import sleep
from env import api_key

In [ ]:
#Josh's code
basegame = 4108071572
url = 'https://americas.api.riotgames.com/'
name = 'NoNoYogi'
response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
puuid =response.json()['puuid']
pulls = 0
count = 1
while pulls != 100:
    
    query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={count}&count=1&api_key={api_key}'
    response = requests.get(url+query)
    
    for game in response.json():
        if int(game[4:]) > basegame:
            print("higher")
        else:
            print('lower')
    count +=1
    pulls +=1

In [ ]:
#Jared's code
def get_new_match_ids(puuids):
    url = 'https://americas.api.riotgames.com/'
    for puuid in puuids:
        i = 1
        while i < 20:
            query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={i-1}&count={i}&api_key={api_key}'
            response = requests.get(url+query)
            match = response.json()
            match_num = int(match[0].strip('NA1_'))
            if match_num > 4108071572:
                i += 1
                match_list.append(match)
            else:
                break

### Combine the Code Above Into One Function

In [ ]:
def get_new_patch_data(name_list, api_key):
    """
    This function takes in a list of usernames that are in the same tier and iterates 
    through them to find their puuids.
    It will then use those puuids to grab all of their match ids that are in the newest patch.
    From there, the function will iterate through all of the match ids and grab the timeline
    data for each match as well as the other game data. 
    Finally, this function will incrementally save the returned json information every 10 times.
    This prevents the loss of all data if the function errors out at some point.
    This function returns nothing.
    """
    
    #Set up the basegame number. Any match_ids below this number are part of the old patch
    #and will not be included. We will use the basegame var for comparisons in the loop
    basegame = 4107000001
    
    #Set up the base url for grabbing match_ids
    match_url = 'https://americas.api.riotgames.com/'
    
    #Set up the base url for grabbing puuids
    puuid_base_url = 'https://na1.api.riotgames.com/'
    
    #Set up another counter to keep track of how many users have been completed
    n = 0
    
    #Set up a counter to keep track of how many json files have been completed in total
    num_json = 0
    
    #Set up one more counter to keep track of how many pulls we have made.
    #Can't go over 100 pulls every 2 minutes.
    num_pulls = 0
    
    #Set up empty lists to store timeline and other game data
    timeline_data = []
    game_data = []
    
    #Loop through each name in the given name_list
    #First grab the puuid, then grab the match_ids, then actual match json info
    for name in name_list:
        
        #for testing
        print(f'Beginning Loop for user {name}')
        
        #Update n var
        n += 1
        
        #Leave a status update after every 100 users
        if n % 100 == 0:
            print(f'Completed {n} users of {len(name_list)}!\n')
        
        ########################## Get puuids ##########################
        
        #Set up the query
        query = f'lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}'
        
        #Get the response
        response = requests.get(puuid_base_url + query)
        
        #Update pull count and sleep for 150 seconds if necessary
        num_pulls += 1
        
        if num_pulls % 100 == 0:
            print('\nHit 100 pulls! Sleeping for 150 seconds.')
            sleep(150)
            print('Continuing!\n')
        
        #Check the response code and leave message if something goes wrong
        if response.status_code != 200:
            print(f'Something went wrong! Status Code: {response.status_code}. Did not get puuid for user: {name}')
            print(f'Skipping this user. There are {len(name_list) - n} users left.\n')
            continue
        
        #Grab the user's puuid
        puuid = response.json()['puuid']
        
        #For testing
        print(f'\t-Got the puuid for user {name}.')
        
        ########################## Get Match IDs #########################
        
        #Now that we have a single puuid, grab all the match ids for that user that are greater
        #than the basegame value. These match_ids will be for the newest patch.
        
        #Set up a counter
        i = 0
        
        #This loop will be set up to run indefinitely. Be sure to break out of it when 
        #Done with the current user
        while i >= 0:
            #Set up the query to get a match_id
            query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={i}&count=1&api_key={api_key}'
            
            #Get the response
            response = requests.get(match_url + query)
            
            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response status code and leave message if something went wrong
            if response.status_code != 200:
                print(f'Something went wrong! Status Code: {response.status_code}. Did not get Match_ID for user: {name}')
                print(f'Skipping this user. There are {len(name_list) - n} users left.\n')
                break
                
            #Grab the match_id
            match = response.json()
            
            #Verify there's something in it
            if len(match) == 0:
                print(f'\t-No match ID for user {name}. Skipping this user!')
                break
            
            #Convert match_id to integer
            match_num = int(match[0].strip('NA1_'))
            
            #Compare current match_id to basegame var.
            #Skip if match_id is less than basegame
            if match_num < basegame:
                print(f'\t-Completed getting all current patch Match IDs for user {name}')
                print(f'\t-There are {len(name_list) - n} users left.')
                break

            ####################### Get Match Info ##########################
            
            #Now that we know the current match_id is part of the newest patch,
            #We can retrieve the actual match info. Keep in mind this is still part
            #of the while loop above
                
            #Every 250 entries, save the lists as json files and reset the lists
            if len(timeline_data) % 250 == 0:    
                #Save timeline data.
                #First, convert it to a df
                timeline_df = pd.DataFrame(timeline_data)

                #Save as json file. Use username string to identify whose file it is
                timeline_df.to_json(f'timeline_data_user_{name}_end_{num_json}.json')

                #Print status message
                print(f'\t-Created timeline_data_user_{name}_end_{num_json}.json')

                #Save other game data
                #First, convert it to a df
                game_df = pd.DataFrame(game_data)

                #Save as json file. Use username string to identify whose file it is
                game_df.to_json(f'other_game_data_user_{name}_end_{num_json}.json')

                #Print status message
                print(f'\t-Created other_game_data_user_{name}_end_{num_json}.json\n')
                
                #Now reset the temp lists
                timeline_data = []
                game_data = []
            
        
            ############################# TIMELINE DATA #######################
            #Set up timeline url
            timeline_query = f'lol/match/v5/matches/{match[0]}/timeline/?api_key={api_key}'

            #Grab timeline json data
            timeline_response = requests.get(match_url + timeline_query)

            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response and leave a status message
            if timeline_response.status_code != 200:
                print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
                print(f'Skipping this match ID.\n')
                continue

            #Turn it into json format
            timeline_json = timeline_response.json()

            #Append this data to the timeline_data list
            timeline_data.append(timeline_json)
            
            ############################# OTHER GAME DATA #######################
            #Set up game data url
            game_query = f'lol/match/v5/matches/{match[0]}?api_key={api_key}'

            #Grab game json data
            game_response = requests.get(match_url + game_query)

            #Update pull count and sleep for 150 seconds if necessary
            num_pulls += 1

            if num_pulls % 100 == 0:
                print('\nHit 100 pulls! Sleeping for 150 seconds.')
                sleep(150)
                print('Continuing!\n')
            
            #Check response and leave a status message.
            if game_response.status_code != 200:
                print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
                print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')

                #Remove the last entry in the timeline_data list
                timeline_data.pop()
                continue

            #Turn it into json format
            game_json = game_response.json()

            #Append this data to the game_data list
            game_data.append(game_json)
            
            #Update num_json
            num_json += 1
            
            #Update i
            i += 1
            
        #Tell the user how many match_ids were used for the current user
        print(f'\t-Completed {i} Match IDs for user {name}\n')
    
    #Save any json data that was not saved at the end of the for loop
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)

    #Save as json file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_user_{name}_end_{num_json}.json')

    #Print status message
    print(f'Created timeline_data_user_{name}_end_{num_json}.json')

    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)

    #Save as json file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_user_{name}_end_{num_json}.json')

    #Print status message
    print(f'Created other_game_data_user_{name}_end_{num_json}.json')
    
    #Complete! Leave a message
    print('All users completed! Follow suit.')
            

In [15]:
#For testing, import the user name
from pro_names import pro_names
from env import api_key
import requests
import pandas as pd
from bs4 import BeautifulSoup
from env import api_key
from time import sleep
from acquire import extract, build_prepared_df, get_new_patch_data, get_players
pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")
import acquire
import re

In [5]:
df1 = pd.read_csv('5451_players.csv')
player_list = df1.Player.to_list()
player_list

['BJNA',
 'BHSc',
 'QAZx',
 'Can Win N0 Need',
 'lT0Tl',
 'NA chopsticks',
 'Hyacintho',
 'Cawwo',
 'LFT RCON',
 'catnip1',
 'Cyrus D',
 'UF Trollzelda',
 'Euphorian',
 'Princess Mico',
 'Jachsparrow',
 'Hero of mischief',
 'i will crush you',
 'HAYES',
 'GraceWangFinal18',
 'BratEvolved',
 'usu',
 'SSAK',
 'Adan',
 'SC itssyaboinate',
 'MeIody',
 'LGM Snow',
 'INSSA HUNTER',
 'Young Tiller',
 'nifty jg',
 'FaceCheckOnBush',
 'LPL x LCK Aggro',
 'Z Z T A ',
 'Youli',
 'buhudong',
 'thats fervor',
 'PLEASEHELPADC',
 'Augustine',
 'ciri of cintra',
 'Exiled Heretic',
 'POTATOOOOOOOOO',
 'Gweiss',
 'Kyusung',
 'TrevorKTran',
 'Xiao Hun zii',
 'Nanjingwaiguoyu',
 'Weaker',
 'MEMEME670',
 'Fanthom',
 'SuperKoolGus',
 'eliron',
 'SKT T1 Kaisaa',
 'o2h',
 'Nice Bunz',
 'WQ only',
 'Bio1',
 'Swiftzzz',
 'smallplaypi',
 'Steam Sale',
 'Elekktro',
 'qwer123456',
 'Two Beginnings',
 'Xiao Suan Nai',
 'Yellow dog',
 'Venaz',
 'Sam123',
 '1186',
 'Spladaddy',
 'Dahyun x Twice',
 'AllGirlsRTheSame',

In [6]:
test_list = player_list[4174:4300]

In [7]:
last_timeline_data, last_game_data = get_new_patch_data(test_list, api_key)

Beginning Loop for user Suna Shogun Sama
	-Got the puuid for user Suna Shogun Sama.
	-Completed getting all current patch Match IDs for user Suna Shogun Sama
	-There are 599 users left.
	-Completed 0 Match IDs for user Suna Shogun Sama

Beginning Loop for user OpenBackpack
	-Got the puuid for user OpenBackpack.
	-Completed getting all current patch Match IDs for user OpenBackpack
	-There are 598 users left.
	-Completed 0 Match IDs for user OpenBackpack

Beginning Loop for user BOBtimers Kitten
	-Got the puuid for user BOBtimers Kitten.
	-Created timeline_data_user_BOBtimers Kitten_end_0.json
	-Created other_game_data_user_BOBtimers Kitten_end_0.json

	-Completed getting all current patch Match IDs for user BOBtimers Kitten
	-There are 597 users left.
	-Completed 8 Match IDs for user BOBtimers Kitten

Beginning Loop for user Xx ZEDGOD69 xX
	-Got the puuid for user Xx ZEDGOD69 xX.
	-Completed getting all current patch Match IDs for user Xx ZEDGOD69 xX
	-There are 596 users left.
	-Comple

	-Completed getting all current patch Match IDs for user SpazzJG
	-There are 563 users left.
	-Completed 0 Match IDs for user SpazzJG

Beginning Loop for user Krustyy
	-Got the puuid for user Krustyy.
	-Completed getting all current patch Match IDs for user Krustyy
	-There are 562 users left.
	-Completed 1 Match IDs for user Krustyy

Beginning Loop for user Hook King
	-Got the puuid for user Hook King.
	-Completed getting all current patch Match IDs for user Hook King
	-There are 561 users left.
	-Completed 3 Match IDs for user Hook King

Beginning Loop for user UmassZhuimeng
Something went wrong! Status Code: 404. Did not get puuid for user: UmassZhuimeng
Skipping this user. There are 560 users left.

Beginning Loop for user Tempos
	-Got the puuid for user Tempos.
	-Completed getting all current patch Match IDs for user Tempos
	-There are 559 users left.
	-Completed 2 Match IDs for user Tempos

Beginning Loop for user Zakn7
	-Got the puuid for user Zakn7.
	-Completed getting all curre

	-Got the puuid for user quantumfan.
	-Completed getting all current patch Match IDs for user quantumfan
	-There are 525 users left.
	-Completed 0 Match IDs for user quantumfan

Beginning Loop for user FobAsian
	-Got the puuid for user FobAsian.
	-Completed getting all current patch Match IDs for user FobAsian
	-There are 524 users left.
	-Completed 0 Match IDs for user FobAsian

Beginning Loop for user 12ad
	-Got the puuid for user 12ad.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user 12ad
	-There are 523 users left.
	-Completed 12 Match IDs for user 12ad

Beginning Loop for user helianwu
	-Got the puuid for user helianwu.
	-Completed getting all current patch Match IDs for user helianwu
	-There are 522 users left.
	-Completed 1 Match IDs for user helianwu

Beginning Loop for user ArticXII
	-Got the puuid for user ArticXII.
	-Completed getting all current patch Match IDs for user ArticXII
	-There are 521 users left.
	-Com

	-Got the puuid for user Murder Mysteries.
	-Created timeline_data_user_Murder Mysteries_end_500.json
	-Created other_game_data_user_Murder Mysteries_end_500.json

	-Completed getting all current patch Match IDs for user Murder Mysteries
	-There are 487 users left.
	-Completed 7 Match IDs for user Murder Mysteries

Beginning Loop for user lil tjay sb
	-Got the puuid for user lil tjay sb.
	-Completed getting all current patch Match IDs for user lil tjay sb
	-There are 486 users left.
	-Completed 0 Match IDs for user lil tjay sb

Beginning Loop for user Trapanese Ramen
	-Got the puuid for user Trapanese Ramen.
	-Completed getting all current patch Match IDs for user Trapanese Ramen
	-There are 485 users left.
	-Completed 0 Match IDs for user Trapanese Ramen

Beginning Loop for user Xz4
	-Got the puuid for user Xz4.
	-Completed getting all current patch Match IDs for user Xz4
	-There are 484 users left.
	-Completed 0 Match IDs for user Xz4

Beginning Loop for user Innominat
	-Got the puui

	-Got the puuid for user Yega.
	-Completed getting all current patch Match IDs for user Yega
	-There are 450 users left.
	-Completed 0 Match IDs for user Yega

Beginning Loop for user Arsh
	-Got the puuid for user Arsh.
	-Completed getting all current patch Match IDs for user Arsh
	-There are 449 users left.
	-Completed 5 Match IDs for user Arsh

Beginning Loop for user JGung
	-Got the puuid for user JGung.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user JGung
	-There are 448 users left.
	-Completed 6 Match IDs for user JGung

Beginning Loop for user Pretty Sexy
	-Got the puuid for user Pretty Sexy.
	-Completed getting all current patch Match IDs for user Pretty Sexy
	-There are 447 users left.
	-Completed 0 Match IDs for user Pretty Sexy

Beginning Loop for user Sword2
	-Got the puuid for user Sword2.
	-Completed getting all current patch Match IDs for user Sword2
	-There are 446 users left.
	-Completed 5 Match IDs for us

...

	-Got the puuid for user ChrisCheon.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user ChrisCheon
	-There are 263 users left.
	-Completed 4 Match IDs for user ChrisCheon

Beginning Loop for user The Conquest
	-Got the puuid for user The Conquest.
	-Completed getting all current patch Match IDs for user The Conquest
	-There are 262 users left.
	-Completed 1 Match IDs for user The Conquest

Beginning Loop for user happens happens
	-Got the puuid for user happens happens.
	-Completed getting all current patch Match IDs for user happens happens
	-There are 261 users left.
	-Completed 23 Match IDs for user happens happens

Beginning Loop for user beo no fear
	-Got the puuid for user beo no fear.
	-Completed getting all current patch Match IDs for user beo no fear
	-There are 260 users left.
	-Completed 0 Match IDs for user beo no fear

Beginning Loop for user Mommy Rodent
	-Got the puuid for user Mommy Rodent.

Hit 100 pulls! Sle

	-Completed getting all current patch Match IDs for user Hot Ruffles2
	-There are 227 users left.
	-Completed 0 Match IDs for user Hot Ruffles2

Beginning Loop for user Carmite
	-Got the puuid for user Carmite.
	-Completed getting all current patch Match IDs for user Carmite
	-There are 226 users left.
	-Completed 21 Match IDs for user Carmite

Beginning Loop for user White Paper
	-Got the puuid for user White Paper.
	-Completed getting all current patch Match IDs for user White Paper
	-There are 225 users left.
	-Completed 0 Match IDs for user White Paper

Beginning Loop for user hlep
	-Got the puuid for user hlep.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user hlep
	-There are 224 users left.
	-Completed 12 Match IDs for user hlep

Beginning Loop for user Moist Smurf
	-Got the puuid for user Moist Smurf.
	-Completed getting all current patch Match IDs for user Moist Smurf
	-There are 223 users left.
	-Completed 2 Match 

	-Got the puuid for user MightyPug.
	-Completed getting all current patch Match IDs for user MightyPug
	-There are 190 users left.
	-Completed 1 Match IDs for user MightyPug

Beginning Loop for user OO9527
	-Got the puuid for user OO9527.
	-Completed getting all current patch Match IDs for user OO9527
	-There are 189 users left.
	-Completed 9 Match IDs for user OO9527

Beginning Loop for user Only BIue
	-Got the puuid for user Only BIue.
	-Completed getting all current patch Match IDs for user Only BIue
	-There are 188 users left.
	-Completed 1 Match IDs for user Only BIue

Beginning Loop for user LYA2021ADC
	-Got the puuid for user LYA2021ADC.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user LYA2021ADC
	-There are 187 users left.
	-Completed 16 Match IDs for user LYA2021ADC

Beginning Loop for user Sundae Gukbap
	-Got the puuid for user Sundae Gukbap.
	-Completed getting all current patch Match IDs for user Sundae Gukbap
	

	-Completed getting all current patch Match IDs for user SpicedKimchi
	-There are 152 users left.
	-Completed 1 Match IDs for user SpicedKimchi

Beginning Loop for user Dizzydish
	-Got the puuid for user Dizzydish.
	-Completed getting all current patch Match IDs for user Dizzydish
	-There are 151 users left.
	-Completed 1 Match IDs for user Dizzydish

Beginning Loop for user relegatedd
	-Got the puuid for user relegatedd.
	-Completed getting all current patch Match IDs for user relegatedd
	-There are 150 users left.
	-Completed 0 Match IDs for user relegatedd

Beginning Loop for user 0 n a t
	-Got the puuid for user 0 n a t.

Hit 100 pulls! Sleeping for 150 seconds.
Continuing!

	-Completed getting all current patch Match IDs for user 0 n a t
	-There are 149 users left.
	-Completed 19 Match IDs for user 0 n a t

Beginning Loop for user makleven
	-Got the puuid for user makleven.
	-Completed getting all current patch Match IDs for user makleven
	-There are 148 users left.
	-Completed 3 

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [8]:
#Build the function
def build_extracted_df(username, path = './', time = 10):
    """
    This function will take in a path for the json files stored in your directory. 
    It will then read all of them into a list, convert them to lists of dicts, and feed
    them into the extract function. The time variable is for the extract function and determines
    at what timeframe the data will be acquired for. The username is only used for naming the
    .csv file at the end. Please use your last name to avoid confusion.
    
    This function returns a dataframe with data extracted for the specified time. 
    It also automatically saves this dataframe as a .csv.
    
    The path variable defaults to the current directory.
    The time variable defaults to the 15 minute mark.
    
    You will need to have os imported to run this function.
    """
    
    #Gather the names of the timeline json files and sort them
    #This will pull all files that start with 'timeline'
    timeline_files = [ x for x in os.listdir(path) if x.startswith("timeline") ]
    timeline_files.sort()
    
    #Gather the names of the other game data json files and sort them
    #This will pull all files that start with 'other'
    other_data_files = [ x for x in os.listdir(path) if x.startswith("other") ]
    other_data_files.sort()
    
    #Will need an empty df to store the final df
    extracted_df = pd.DataFrame()
    
    #Verify that the lists are the same length
    if len(timeline_files) != len(other_data_files):
        print('Something went wrong! The json lists are not equal!')
        print('Stopping function.')
        return

    for i in range(0, len(timeline_files)):
        #Need to read both of them into dataframes and then convert to list of dicts
        timeline_list = pd.read_json(path + timeline_files[i])
        game_list = pd.read_json(path + other_data_files[i])

        #Convert to list of dicts
        timeline_list = timeline_list.to_dict(orient = 'records')
        game_list = game_list.to_dict(orient = 'records')

        #Now feed them into Joshua C's prepare file. Extract data for the specified timeframe
        temp_df = acquire.extract(timeline_list, game_list, time)

        #Now append the temp_df to the complete_df
        extracted_df = extracted_df.append(temp_df, ignore_index = True)
            
    #Drop duplicates and return the prepared df
    extracted_df = extracted_df.drop_duplicates()
    
    #Now save the extracted_df as a .csv
    extracted_df.to_csv(f'new_extracted_data_{username}.csv', index = False)
    
    return extracted_df

In [18]:
import os
df = build_extracted_df(username, path = './', time = 10)

Skipping: 0 due to <10 min or not classic
Skipping: 1 due to <10 min or not classic
Skipping: 2 due to <10 min or not classic
Skipping: 3 due to <10 min or not classic
Skipping: 4 due to <10 min or not classic
Skipping: 5 due to <10 min or not classic
Finished with: 6 of 249
Finished with: 7 of 249
Skipping: 8 due to <10 min or not classic
Finished with: 9 of 249
Finished with: 10 of 249
Skipping: 11 due to <10 min or not classic
Finished with: 12 of 249
Skipping: 13 due to <10 min or not classic
Finished with: 14 of 249
Skipping: 15 due to <10 min or not classic
Finished with: 16 of 249
Finished with: 17 of 249
Skipping: 18 due to <10 min or not classic
Finished with: 19 of 249
Finished with: 20 of 249
Finished with: 21 of 249
Finished with: 22 of 249
Skipping: 23 due to <10 min or not classic
Skipping: 24 due to <10 min or not classic
Skipping: 25 due to <10 min or not classic
Skipping: 26 due to <10 min or not classic
Skipping: 27 due to <10 min or not classic
Skipping: 28 due to <1

Finished with: 231 of 249
Finished with: 232 of 249
Finished with: 233 of 249
Finished with: 234 of 249
Finished with: 235 of 249
Skipping: 236 due to <10 min or not classic
Skipping: 237 due to <10 min or not classic
Skipping: 238 due to <10 min or not classic
Skipping: 239 due to <10 min or not classic
Skipping: 240 due to <10 min or not classic
Skipping: 241 due to <10 min or not classic
Finished with: 242 of 249
Skipping: 243 due to <10 min or not classic
Skipping: 244 due to <10 min or not classic
Skipping: 245 due to <10 min or not classic
Skipping: 246 due to <10 min or not classic
Finished with: 247 of 249
Finished with: 248 of 249
Finished with: 249 of 249
Done! now lets get some korean bbq, more chicken plz! haha....follow suit
Finished with: 0 of 249
Finished with: 1 of 249
Finished with: 2 of 249
Skipping: 3 due to <10 min or not classic
Skipping: 4 due to <10 min or not classic
Skipping: 5 due to <10 min or not classic
Finished with: 6 of 249
Skipping: 7 due to <10 min or 

Finished with: 208 of 249
Finished with: 209 of 249
Skipping: 210 due to <10 min or not classic
Skipping: 211 due to <10 min or not classic
Skipping: 212 due to <10 min or not classic
Skipping: 213 due to <10 min or not classic
Finished with: 214 of 249
Finished with: 215 of 249
Skipping: 216 due to <10 min or not classic
Finished with: 217 of 249
Finished with: 218 of 249
Skipping: 219 due to <10 min or not classic
Skipping: 220 due to <10 min or not classic
Skipping: 221 due to <10 min or not classic
Skipping: 222 due to <10 min or not classic
Finished with: 223 of 249
Skipping: 224 due to <10 min or not classic
Finished with: 225 of 249
Finished with: 226 of 249
Finished with: 227 of 249
Finished with: 228 of 249
Finished with: 229 of 249
Skipping: 230 due to <10 min or not classic
Skipping: 231 due to <10 min or not classic
Skipping: 232 due to <10 min or not classic
Skipping: 233 due to <10 min or not classic
Skipping: 234 due to <10 min or not classic
Skipping: 235 due to <10 min

Finished with: 189 of 249
Finished with: 190 of 249
Finished with: 191 of 249
Finished with: 192 of 249
Finished with: 193 of 249
Skipping: 194 due to <10 min or not classic
Finished with: 195 of 249
Finished with: 196 of 249
Finished with: 197 of 249
Finished with: 198 of 249
Finished with: 199 of 249
Finished with: 200 of 249
Finished with: 201 of 249
Finished with: 202 of 249
Skipping: 203 due to <10 min or not classic
Finished with: 204 of 249
Finished with: 205 of 249
Finished with: 206 of 249
Skipping: 207 due to <10 min or not classic
Finished with: 208 of 249
Finished with: 209 of 249
Skipping: 210 due to <10 min or not classic
Skipping: 211 due to <10 min or not classic
Skipping: 212 due to <10 min or not classic
Finished with: 213 of 249
Finished with: 214 of 249
Finished with: 215 of 249
Finished with: 216 of 249
Skipping: 217 due to <10 min or not classic
Finished with: 218 of 249
Finished with: 219 of 249
Skipping: 220 due to <10 min or not classic
Finished with: 221 of 24

KeyboardInterrupt: 

In [19]:
df.to_csv('min_10_start_3700_end_4173_.csv')

In [ ]:
df.info(verbose = True)

# Get the New Data

Now that the function has been completed, get the desired data. Joshua C. is gathering the data for the pro players. I will get the data using the names from Jared's list.

In [ ]:
#Load the names csv
name_list = pd.read_csv('2640_players.csv', index_col = [0])
name_list

__How Many?__

Since there are over 2600 names, I will only attempt to gather for the first 1000 of them. I will save a json file after every 250 json entries.

In [ ]:
name_list = list(name_list.players)

In [ ]:
name_list = name_list[:1000]

In [ ]:
#Run the function
get_new_patch_data(name_list, api_key)

In [ ]:
#Completed 320 names in name_list before it errored out.
#Create new list starting at index 321 and restart the function
name_list = name_list[321:1000]

In [ ]:
#Run the function
get_new_patch_data(name_list, api_key)

# Join All of the New Json Files Together

In [ ]:
#Import os and try to programatically grab the json file names
import os

In [ ]:
timeline_files = [ x for x in os.listdir('./new_data') if x.startswith("timeline") ]
timeline_files.sort()
timeline_files

In [ ]:
other_data_files = [ x for x in os.listdir('./new_data') if x.startswith("other") ]
other_data_files.sort()
other_data_files

In [ ]:
import pandas as pd
import acquire
import prepare

In [ ]:
#Create a function to loop through these files and prepare them
#Will need an empty df to store the final df
complete_df = pd.DataFrame()

for i in range(0, len(timeline_files[:2])):
    #Need to read both of them into dataframes and then convert to list of dicts
    timeline_list = pd.read_json('./new_data/' + timeline_files[i])
    game_list = pd.read_json('./new_data/' + other_data_files[i])

    #Convert to list of dicts
    timeline_list = timeline_list.to_dict(orient = 'records')
    game_list = game_list.to_dict(orient = 'records')

    #Now feed them into Joshua C's prepare file. Extract data at the 15 minute mark
    temp_df = acquire.extract(timeline_list, game_list, 15)
    print(f'Extracted the data! Entries: {len(temp_df)}')

    #Now prepare the data
    temp_df = prepare.prepare(temp_df)
    print(f'Prepared the data! Entries: {len(temp_df)}')

    #Now append the temp_df to the complete_df
    complete_df = complete_df.append(temp_df, ignore_index = True)
    print(f'Appended the data! Complete DataFrame Entries: {len(complete_df)}')

#Drop duplicates and return the prepared df
complete_df = complete_df.drop_duplicates()

In [ ]:
complete_df.info()

In [ ]:
#Build the function
def build_extracted_df(username, path = './', time = 15):
    """
    This function will take in a path for the json files stored in your directory. 
    It will then read all of them into a list, convert them to lists of dicts, and feed
    them into the extract function. The time variable is for the extract function and determines
    at what timeframe the data will be acquired for. The username is only used for naming the
    .csv file at the end. Please use your last name to avoid confusion.
    
    This function returns a dataframe with data extracted for the specified time. 
    It also automatically saves this dataframe as a .csv.
    
    The path variable defaults to the current directory.
    The time variable defaults to the 15 minute mark.
    
    You will need to have os imported to run this function.
    """
    
    #Gather the names of the timeline json files and sort them
    #This will pull all files that start with 'timeline'
    timeline_files = [ x for x in os.listdir(path) if x.startswith("timeline") ]
    timeline_files.sort()
    
    #Gather the names of the other game data json files and sort them
    #This will pull all files that start with 'other'
    other_data_files = [ x for x in os.listdir(path) if x.startswith("other") ]
    other_data_files.sort()
    
    #Will need an empty df to store the final df
    extracted_df = pd.DataFrame()
    
    #Verify that the lists are the same length
    if len(timeline_files) != len(other_data_files):
        print('Something went wrong! The json lists are not equal!')
        print('Stopping function.')
        return

    for i in range(0, len(timeline_files)):
        #Need to read both of them into dataframes and then convert to list of dicts
        timeline_list = pd.read_json(path + timeline_files[i])
        game_list = pd.read_json(path + other_data_files[i])

        #Convert to list of dicts
        timeline_list = timeline_list.to_dict(orient = 'records')
        game_list = game_list.to_dict(orient = 'records')

        #Now feed them into Joshua C's prepare file. Extract data for the specified timeframe
        temp_df = acquire.extract(timeline_list, game_list, time)

        #Now append the temp_df to the complete_df
        extracted_df = extracted_df.append(temp_df, ignore_index = True)
            
    #Drop duplicates and return the prepared df
    extracted_df = extracted_df.drop_duplicates()
    
    #Now save the extracted_df as a .csv
    extracted_df.to_csv(f'new_extracted_data_{username}.csv', index = False)
    
    return extracted_df

In [ ]:
#Test the function and build an extracted df that we can feed into the prepare function
extracted_data = build_extracted_df(username = 'smith', path = './new_data/', time = 15)

In [ ]:
extracted_data